# Applying Blockchain to IoT 
### Done by: Sumaya Altamimi

Recent explosion of interest around block chain technology, high demanding IoT devices, the importance, need, and future direction in these fields one may think  whether they make a good fit for the Internet of Things (IoT) sector and how to combine them to achieve perfect secrecy, or at least a more robust security in IoT. The combination of blockchain and IoT might be powerful and might be a good start to more significant businesses and models. There might be several research approaches that have a huge impact for the future of the IoT security. However, because of the success of Bitcoin, and the fact that it is expected for the blockchain (BC) to drive economic change on a global scale, I found BC is so attractive to dives in and apply in IOT. It is expected for the Blockchain to solve a single point of failure.

## Table of contents

1. [Hash function and mining](#mining)
3. [Device Simulation](#device)
4. [Processes](#process)
5. [Blocks](#block)
8. [Differences with Blockchain in Bitcoin](#differences)
9. [References](#references)

In [1]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
%matplotlib inline
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

# Hash function and mining <a name="mining"></a>

So we'll start a little backward and start with the miner implementation.
For the example here, we'll use the SHA256 hash function because it's readily implemented in python. Note that bitcoin uses [two round of SHA256](https://en.bitcoin.it/wiki/Hashcash) instead of one.

So our hash function will turn a string of arbitrary length into a fixed-length string of 64 hexadecimal characters. For example :

In [2]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

def iot_hash(message):
    """
    Returns an hexadecimal hash
    """
    return sha256(message)


def mine(message, difficulty=1):
    """
    Given an input string, will return a nonce such that
    hash(string + nonce) starts with `difficulty` ones
    
    Returns: (nonce, niters)
        nonce: The found nonce
        niters: The number of iterations required to find the nonce
    """
    assert difficulty >= 1, "Difficulty of 0 is not possible"
    i = 0
    prefix = '1' * difficulty
    while True:
        nonce = str(i)
        digest = iot_hash(message + nonce)
        if digest.startswith(prefix):
            return nonce, i
        i += 1

### Test

The more you increase the number of leading ones you require, the harder it becomes (on average) to find a nonce. In bitcoin, this is called the mining difficulty. Note that bitcoin doesn't require a number of leading digits, but instead requires the hash to be below a certain value. But it's the same idea.

So let's define two functions that we'll reuse later : one to hash a string and one to mine a nonce for a given string.

Now, the process of *mining* is : given an arbitrary string $x$, find a nonce such that $hash(x + nonce)$ produces a hash starting with a number of leading ones.

For example here, we'll "mine" a nonce such that the hash of our message ("hello bitcoin") when concatenated with our nonce will have at least 2 leading ones.

In [41]:
message = 'IoT-BC'
for nonce in range(1000):
    digest = sha256(message + str(nonce))
    if digest.startswith('11'):
        print('Found nonce = %d' % nonce)
        break
print(sha256(message + str(nonce)))

Found nonce = 86
1141c060951047f3618234d08334ba76c86d4b757caac2cbeebbe1925d67a921


Mine nonce of varied difficulty :

In [42]:
nonce, niters = mine('IoT-BC', difficulty=1)
print('Took %d iterations' % niters)

nonce, niters = mine('IoT-BC', difficulty=3)
print('Took %d iterations' % niters)

Took 12 iterations
Took 277 iterations


As you can see in this example, the number of iterations required for a difficulty of 3 is much larger than for a difficulty of 1. Note though that you could get lucky and have a string where the first nonce (0 in our case) would yield the solution. So the difficulty controls the *average* number of tries. We can do a nice plot of that

# Device simulation <a name="device"></a>

Each IoT device should has a private/public key pair. <br>
 
The public key is used to receive data and the private key is used to process data.

By signing a message with our private key,anybody else can verify the signature using our public key.

Note that the actual blockchain is more complicated.

A blockchain is a set of multiple private/public key pairs, and an address is not directly the public key. 

This ensures better privacy and security, but for IoT implementation proposal, we'll use a single key and use the public key as the address.

In [5]:
class Device(object):
    """
    A device has a private/public key pair
    """
    def __init__(self):
        random_gen = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random_gen)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)
        
    @property
    def address(self):
        """We take a shortcut and say address is public key"""
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    
    def sign(self, message):
        """
        Sign a message with this wallet
        """
        h = SHA.new(message.encode('utf8'))
        return binascii.hexlify(self._signer.sign(h)).decode('ascii')
    

In [6]:
def verify_signature(device_address, message, signature):
    """
    Check that the provided `signature` corresponds to `message`
    signed by the device at `device_address`
    """
    pubkey = RSA.importKey(binascii.unhexlify(device_address))
    verifier = PKCS1_v1_5.new(pubkey)
    h = SHA.new(message.encode('utf8'))
    return verifier.verify(h, binascii.unhexlify(signature))

### Test

In [7]:
# Check that the wallet signing functionality works
d1 = Device()
msg = 'hey'
signature = d1.sign(msg)
assert verify_signature(d1.address, msg, signature)
assert not verify_signature(d1.address, 'rogue message', signature)

# Process of Sending Messages <a name="process"></a>

To exchange messages between devices, we'll use Process. 
<br> A Process is composed of :
- A **sender**, who will write and  sign  the message.
- A **number of inputs**, which are other messages outputs. <br>
The recipient of all those should be the sender's device.
Otherwise you could see other people's data.
- A **number of outputs**, each of which specify an **message** and a **recipient**

In [8]:
class ProcessInput(object):
    """
    An input for a Process. This points to an output of another process.
    """
    def __init__(self, process, output_index):
        self.process = process
        self.output_index = output_index
        assert 0 <= self.output_index < len(process.outputs)
        
    def to_dict(self):

        d = {
            'process': self.process.hash(),
            'output_index': self.output_index
        }
        return d
    
    @property
    def parent_output(self):
        return self.process.outputs[self.output_index]
    

In [9]:
class ProcessOutput(object):
    """
    An output for a Process. This specifies the message and a recipient device
    """
    def __init__(self, recipient_address, message):
        self.recipient = recipient_address
        self.message = message
        
    def to_dict(self):
        d = {
            'recipient_address': self.recipient,
            'message': self.message
        }
        return d

In [10]:
class Process(object):
    def __init__(self, device, inputs, outputs):
        """
        Create a process of sending message from the provided device
        """
        self.inputs = inputs
        self.outputs = outputs
        self.signature = device.sign(json.dumps(self.to_dict(include_signature=False)))
        
    def to_dict(self, include_signature=True):
        d = {
            "inputs": list(map(ProcessInput.to_dict, self.inputs)),
            "outputs": list(map(ProcessOutput.to_dict, self.outputs))
        }
        if include_signature:
            d["signature"] = self.signature
        return d
    
    def hash(self):
        return iot_hash(json.dumps(self.to_dict()))

Since all processes need a parent, we also need a root in our hierarchy.<br>
This will be the **RootProcess**.

In [11]:
class RootProcess(Process):
    
    """
    This is the first process. which is a special process
    with no input and 'first process' message output
    """
    def __init__(self, recipient_address, message='first process'):
        self.inputs = []
        self.outputs = [
        ProcessOutput(recipient_address, message)
        ]
        self.signature = 'genesis'
        
    def to_dict(self, include_signature=False):
        # TODO: Instead, should sign root process with well-known public key ?
        assert not include_signature, "Cannot include signature of root process"
        return super().to_dict(include_signature=False)

### Make process between alice, bob and walter

In [12]:
alice = Device()
bob = Device()
walter = Device()

In [13]:
p1 = RootProcess(alice.address,'I am Alice')

p2 = Process(alice, [ProcessInput(p1, 0)], [ProcessOutput(bob.address, 'Hello Bob')])
# Alice --Hi --> Bob
#       -- Hi --> Walter
p3 = Process( alice, [ProcessInput(p1, 0)],
    [ProcessOutput(bob.address, 'Hi Bob'), ProcessOutput(walter.address, 'Hi Walter')]
)

# # Walter -- Hi --> Bob
# indexes : which process sent me ? and in which order I listed in that process?
p4 = Process(walter, [ProcessInput(p3, 1)],[ProcessOutput(bob.address, 'Hi Bob')])

## Bob -- Hi --> Walter
p5 = Process( 
    bob,[ProcessInput(p2, 0), ProcessInput(p3, 0),ProcessInput(p4, 0)],
    [ProcessOutput(walter.address, 'Hi Walter')]
)

processes = [p1, p2, p3, p4 , p5]

In blockchain, you never store all messages in your device.<br>
Instead, you go through the whole chain of proceesses to view all your messages .<br> Let's write a function to do that

In [14]:
def view_messages(device_address, processes):
    """
    Given an address and a list of processes,
    view messages of the address
    """
    messages = []
    for p in processes:
        for prout in p.outputs:
            if prout.recipient == device_address:
                messages.append(prout.message)
    return messages

### Test

In [15]:
print("Alice  has messages", view_messages(alice.address, processes),'\n')
print("Bob    has messages" , view_messages(bob.address, processes),'\n')
print("Walter has messages" , view_messages(walter.address, processes),'\n')

Alice  has messages ['I am Alice'] 

Bob    has messages ['Hello Bob', 'Hi Bob', 'Hi Bob'] 

Walter has messages ['Hi Walter', 'Hi Walter'] 



#### We also want to be able to verify that a process is valid. This means :

**All messages from a device address are sent by that device address.** <br>
This means checking that:
- all inputs are owned by the device's owner
- That the process is signed by the owner of the device

In [16]:
def verify_process(process):
    
    pr_message = json.dumps(process.to_dict(include_signature=False)) #outputs & inputs
    
    if isinstance(process, RootProcess):
        # TODO: We should probably be more careful about validating first process
        return True

    # (1) Verify input processes
    for px in process.inputs:
        if not verify_process(px.process):
            logging.error("Invalid parent process")
            return False
        
    first_input_address = process.inputs[0].parent_output.recipient    
    for prin in process.inputs[1:]:
        if prin.parent_output.recipient != first_input_address:
            logging.error(
                "Process inputs belong to multiple devices (%s and %s)" %
                (prin.parent_output.recipient, first_input_address)
            )
        return False
       
    # (ii) Verify signature
    if not verify_signature(first_input_address, pr_message, process.signature):
        logging.error("Invalid process signature, trying to send by someone else's address ?")
        return False
    

    return True

#### Will make invalid process p3:
**Bob** is trying to send using **Alice**'s 
, **Alice** was the recipient of the input - p1

In [17]:
p1 = RootProcess(alice.address,'Actual Alice')

p2 = Process(alice, [ProcessInput(p1, 0)], [ProcessOutput(walter.address, 'Hi walter')])

p3 = Process(bob, [ProcessInput(p1, 0)],[ProcessOutput(walter.address, 'Hi walter')])

processes = [p1, p2, p3]

## Test

In [18]:
assert verify_process(p1)

In [19]:
assert verify_process(p2)

In [20]:
assert verify_process(p3)

ERROR:root:Invalid process signature, trying to send by someone else's address ?


AssertionError: 

# Putting Processes in blocks <a name="block"></a>

Now that we have :

- A way to define a device (as a private/public key pair)
- A way to create process between devices
- A way to verify processes (by checking the signature matches)

What remains is to group processes into blocks and have miners mine blocks. <br>
Mining a block consists of two parts :
- Verifying the processes in the block
- Finding a nonce such that the block's hash starts with a number of 0

In [21]:
DIFFICULTY = 2

In [22]:
class Block(object):
    
    def __init__(self, current_block_processes, previous_block, miner_device_address, skip_verif=False):

        self.current_block_processes = [RootProcess(miner_device_address)] + current_block_processes
        self.previous_block = previous_block
        
        if not skip_verif:
            assert all(map(verify_process, current_block_processes))
        
        json_block = json.dumps(self.to_dict(include_hash=False))
        self.nonce, _ = mine(json_block, DIFFICULTY)
        self.hash = iot_hash(json_block + self.nonce)
    
    
    def to_dict(self, include_hash=True):
        d = {
            "processes": list(map(Process.to_dict, self.current_block_processes)),
            "previous_block": self.previous_block.hash,
        }
        if include_hash:
            d["nonce"] = self.nonce
            d["hash"] = self.hash
        return d

In [23]:
class FirstBlock(Block):
    #This is the only block with no previous block
    
    def __init__(self, miner_device_address):
        super(FirstBlock, self).__init__(current_block_processes=[], previous_block=None, miner_device_address=miner_device_address)

    def to_dict(self, include_hash=True):
        d = {
            "processes": [],
            "first_block": True,
        }
        if include_hash:
            d["nonce"] = self.nonce
            d["hash"] = self.hash
        return d

#### Verify blocks :
Verifies that a block is valid :
- (1) Verifies the hash **starts with** the required amount of **ones** (2 in our case)
- (2) Verifies **all processes** are **valid**
- (3) Verifies the **first process** in the block is a **RootProcess** 

In [24]:
def verify_block(block, first_block, used_outputs=None):
    
    """
    Args:
    # block : The block to validate
    # First_block is the first block which needs to be shared by everybody.
    # E.g. hardcoded somewhere
    # used_outputs: list of outputs used in processes for all blocks above this one
    """
    if used_outputs is None:
        used_outputs = set()
    
    # (1) Verify hash 
    prefix = '1' * DIFFICULTY
    if not block.hash.startswith(prefix):
        logging.error("Block hash (%s) doesn't start with prefix %s" % (block.hash, prefix))
        return False
    
    # (2.i) Processes are valid
    if not all(map(verify_process, block.current_block_processes)):
        return False

    
    # (2.ii) Verify previous blocks up to the first block
    if not (block.hash == first_block.hash):
        if not verify_block(block.previous_block, first_block, used_outputs):
            logging.error("Failed to validate ancestor block")
            return False
    
    # (3.i) Verify the first process is the RootProcess
    pr0 = block.current_block_processes[0]
    if not isinstance(pr0, RootProcess):
        logging.error("Process 0 is not a RootProcess")
        return False

    
    # (3.ii) Only the first process shall be a RootProcess
    for i, pr in enumerate(block.current_block_processes):
        if i == 0:
            if not isinstance(pr, RootProcess):
                logging.error("Non-Root Process at index 0")
                return False  
        elif isinstance(pr, RootProcess):
            logging.error("RootProcess (hash=%s) at index %d != 0", pr.hash(), i)
            return False
    return True

## Test 

In [25]:
alice = Device()
bob = Device()
walter = Device()

In [26]:
first_block = FirstBlock(miner_device_address=alice.address) # Alice mined 1 block 
p1 = first_block.current_block_processes[0] #existing process (root)

In [27]:
p2 = Process(
    alice,
    [ProcessInput(p1, 0)],
    [ProcessOutput(bob.address, 'Hi Bob'),  ProcessOutput(walter.address, 'Hi walter')]
)

p3 = Process(
    walter,
    [ProcessInput(p2, 1)], 
    [ProcessOutput(bob.address, 'Hello Bob')])

p4 = Process(
    bob, 
    [ProcessInput(p2, 0)],
    [ProcessOutput(walter.address, 'Hello walter')]
)

In [28]:
block1 = Block([p2,p3], previous_block=first_block, miner_device_address=bob.address)
## Bob mined 1 block
block2 = Block([p4], previous_block=block1, miner_device_address=walter.address) 
## Walter mined 1 block 

In [31]:
print(" First block  : " + first_block.hash)
print("block1        : " + block1.hash)
print("block2        : " + block2.hash)

 First block  : 11658800373e1d8ae2237a4d216a1ed03553d1f5e0dac5c373808998c8af52f5
block1        : 11aaca04eee9096d378b93c86a0cf9b10e9cda7a4a1e036715ead7ca3deb14e9
block2        : 11c3b530b05c08cc7bada56681353fc165c4bcb23af5aaa62a48b110d79305fb


In [32]:
verify_block(first_block, first_block)

True

In [33]:
verify_block(block1, first_block)

True

In [34]:
verify_block(block2, first_block)

True

#### collect

In [37]:
def process_chain(block, first_block):
    """Recursively collect processes in `block` & all 
    of its previous blocks to simulate BC of messages"""

    processes = [] + block.current_block_processes
    if block.hash != first_block.hash:
        processes += process_chain(block.previous_block, first_block)
    
    return processes

### test

In [38]:
processes = process_chain(block2, first_block)
processes

In [39]:
print("Alice has %s messages" % view_messages(alice.address, processes))
print("Walter has %s messages" % view_messages(walter.address, processes))
print("Bob has %s messages" % view_messages(bob.address, processes))

Alice has ['first process'] messages
Walter has ['first process', 'Hello walter', 'Hi walter'] messages
Bob has ['first process', 'Hi Bob', 'Hello Bob'] messages


# Main differences with real blockchain <a name="differences"></a>

Our goal is not to have a bitcoin implementation, nor conformant blockchain for IoT. <br>
Many things are not implemented, The most important are :

- This uses the wallet public key as its address. Bitcoin uses [more complicated addresses](https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses)
- Bitcoin uses ECDSA for signature, this uses RSA.
- Bitcoin uses two rounds of SHA256, this uses one
- Bitcoin transactions use a [scripting language](https://en.bitcoin.it/wiki/Script). Here we just have
  simple transactions with one amount per output
- Bitcoin requires a way to broadcast blocks to all the nodes. The whole communication part is left out from this    notebook.
  See the [Client Node Discovery](https://en.bitcoin.it/wiki/Satoshi_Client_Node_Discovery) and [Protocol documentation](https://en.bitcoin.it/wiki/Protocol_documentation) pages on the bitcoin wiki
- Bitcoin uses a [Merkle tree](https://en.wikipedia.org/wiki/Merkle_tree) to save disk space by pruning spent   
  transactions.
  

## Security issues

In addition to the differences above, there are a number of security issues with this implementation :
- Use of floating point arithmetic can lead to many problems (e.g. loss of precisions).
- The JSON serialization as implemented here is not reproducible across platforms.<br> For example, two serialization of the same transaction on two different platforms could produce two different JSON (line endings, keys ordering,   spaces).
- The mine method implemented here can loop forever due to integer overflow.

## References <a name = 'references'></a>

- [Bhavani Shankar](https://github.com/bshankar)
- [original bitcoin](https://bitcoin.org/bitcoin.pdf) 
- [Michael Nielsen's bitcoin explanation](http://www.michaelnielsen.org/ddi/how-the-bitcoin-protocol-actually-works/).
- [bitcoin wiki](https://en.bitcoin.it/wiki/Main_Page)
